# Homework 4:

Below is a link to data collected over many years by Carmen Reinhart (with her coauthors Ken Rogoff, Christoph Trebesch, and Vincent Reinhart). These include Banking Crisis dates for more than 70 countries from 1800-present, exchange rate crises, stock market crises, sovereign debt growth and default, and many other data series. 
Please download here:

https://www.hbs.edu/behavioral-finance-and-financial-stability/data/Pages/global.aspx

For the following, please show me that you know how to use lambda and map functions where possible and efficient.

Q1: Read the data from the excel file into a pandas data frame:

In [1]:
import pandas as pd
banking_crisis_raw = pd.read_excel("20160923_global_crisis_data.xlsx")

Q2. From the dataset, create a new dataframe. Using a lambda function, select only those incidents where the annual inflation rate was 25% or higher. The dataframe should have three columns, Country Name, Year and Inflation rate. 


In [2]:
# Rename columns and convert "Inflation rate" to numeric
banking_crisis_raw = banking_crisis_raw.rename(columns={
    "Inflation, Annual percentages of average consumer prices": "Inflation rate", 
    "Country": "Country Name"
})
banking_crisis_raw["Inflation rate"] = pd.to_numeric(banking_crisis_raw["Inflation rate"], errors="coerce")

In [3]:
# Select relevant columns and filter for inflation rate >= 25%
banking_crisis = banking_crisis_raw[["Country Name", "Year", "Inflation rate"]]
banking_crisis = banking_crisis[banking_crisis["Inflation rate"].apply(lambda x: x >= 25)]
print(banking_crisis)

      Country Name    Year  Inflation rate
78         Algeria  1877.0    2.911605e+01
143        Algeria  1942.0    2.812500e+01
144        Algeria  1943.0    4.634146e+01
145        Algeria  1944.0    4.166667e+01
146        Algeria  1945.0    2.941176e+01
...            ...     ...             ...
15178     Zimbabwe  2004.0    1.327468e+02
15179     Zimbabwe  2005.0    5.858444e+02
15180     Zimbabwe  2006.0    1.281114e+03
15181     Zimbabwe  2007.0    6.627989e+04
15182     Zimbabwe  2008.0    2.198970e+07

[843 rows x 3 columns]


Q3. Write code to tell me how many countries experienced 25% inflation or higher?

In [4]:
num_countries = banking_crisis["Country Name"].nunique()
print(num_countries)

61


Q4. How many years across all countries were there events of 25% or higher inflation?

In [5]:
num_years = banking_crisis["Year"].nunique()
print(num_years)

175


Q5. Of the years where inflaton was 25% or higher, how many of those years resulted in a systemic crisis? 

In [6]:
# Merge the original and filtered datasets to find overlapping incidents with an inflation rate of 25% or higher
overlapped_incidents = pd.merge(banking_crisis_raw, banking_crisis, on=["Country Name", "Year", "Inflation rate"])

# Filter for incidents where "Systemic Crisis" equals 1, then count unique years
years_systemic_crisis = overlapped_incidents[overlapped_incidents["Systemic Crisis"] == 1]["Year"].nunique()
print(years_systemic_crisis)

36


Q6. What were the countries and years associated with the total from Q5?

In [7]:
systemic_crisis = overlapped_incidents[overlapped_incidents["Systemic Crisis"] == 1]
print(systemic_crisis[["Year", "Country Name"]])

       Year Country Name
7    1991.0      Algeria
8    1992.0      Algeria
50   1890.0    Argentina
69   1980.0    Argentina
70   1981.0    Argentina
..      ...          ...
838  2004.0     Zimbabwe
839  2005.0     Zimbabwe
840  2006.0     Zimbabwe
841  2007.0     Zimbabwe
842  2008.0     Zimbabwe

[109 rows x 2 columns]


Q7. At times of civil unrest or war, what was the average interest rate?

In [8]:
# Identify incidents of civil unrest or war
civil_unrest = banking_crisis_raw["Defaults_External_Notes"].str.contains("civil unrest", case=False)
war = banking_crisis_raw["Defaults_External_Notes"].str.contains("war", case=False)

# Filter dataset for rows related to civil unrest or war
unrest_war_incidents = banking_crisis_raw[civil_unrest | war]

# Calculate the average inflation rate for these incidents
average_inflation_rate = unrest_war_incidents["Inflation rate"].mean()
print(average_inflation_rate)

2.272305982621385e+24


Q8: Create a new dataframe that lists the top five countries with the average highest interest rates historically, 
with a column for the country, and the average rate.

In [9]:
# Calculate average inflation rates by country
average_rates = banking_crisis_raw.groupby("Country Name")["Inflation rate"].mean().reset_index()

# Sort and select top five countries by average rate
top_five_average_rates = average_rates.sort_values(by="Inflation rate", ascending=False).head(5)

# Rename columns
top_five_average_rates.columns = ["Country", "Average Rate"]

print(top_five_average_rates)

      Country  Average Rate
27    Hungary  1.035976e+25
24     Greece  1.650901e+08
22    Germany  1.023972e+08
69   Zimbabwe  2.371989e+05
43  Nicaragua  3.920163e+02


Q9: Create a new dataframe that lists the countries and years where there was "negative inflation".
What does negative inflation mean, and why did it occur?

In [10]:
negative_inflation_incidents = banking_crisis_raw[banking_crisis_raw["Inflation rate"] < 0][["Country Name", "Year"]]
print(negative_inflation_incidents)

      Country Name    Year
73         Algeria  1872.0
75         Algeria  1874.0
76         Algeria  1875.0
77         Algeria  1876.0
79         Algeria  1878.0
...            ...     ...
15112     Zimbabwe  1938.0
15183     Zimbabwe  2009.0
15188     Zimbabwe  2014.0
15189     Zimbabwe  2015.0
15190     Zimbabwe  2016.0

[2301 rows x 2 columns]


<span style="color:blue"> Deflation indicates a decline in prices for goods and services and the purchasing power of currency rises. Deflation is typically caused by a change in government policy and consumers' reactions to it. (1) The Federal Reserve may deploy a tight monetary policy, pulling back on spending and raising interest rates. This action makes it harder for people to borrow money to buy goods and services. (2) When demand falls, businesses may lower prices to encourage people to spend. (3) Technological advances can help businesses produce more goods at a lower cost, increasing the supply on the shelves. Source: https://www.empower.com/the-currency/money/deflation
</span>

## REDUCE
reduce applies a provided function to the items of an iterable (e.g., a list), two items at a time, reducing the iterable to a single value. It takes two arguments: a function and an iterable. Optionally, it can take an initializer as a third argument. The function it takes as the first argument needs to accept two inputs (we can think of them as the accumulator and the current item).

Let's consider calculating the total GDP (Gross Domestic Product) of a list of countries, where each country's GDP is represented as a number in a list. The reduce function will be used to sum up these GDP values to find the total GDP.

In [11]:
from functools import reduce

# List of GDP values for a set of countries (in trillions of USD)
gdp_values = [21.43, 14.34, 5.15, 4.88, 3.86]  # Example values for USA, China, Japan, Germany, India, etc.

# Function to sum two values
def sum_gdp(x, y):
    return x + y

# Use reduce to calculate the total GDP
total_gdp = reduce(sum_gdp, gdp_values)

'''gdp_values represents a list of GDPs for different countries.
The sum_gdp function is used to sum two GDP values.
reduce iterates through the gdp_values list, continuously applying sum_gdp to accumulate the total GDP.'''

print("Total GDP:", total_gdp, "trillion USD")

Total GDP: 49.66 trillion USD


Q10.  Now rewrite as a lambda function


In [12]:
total_gdp = reduce(lambda x, y: x + y, gdp_values)
print("Total GDP:", total_gdp, "trillion USD")

Total GDP: 49.66 trillion USD


Q11. 
Which are the top three countries that experienced the biggest change in exchange rate with the USD from 2007-2008?
Defines a function calculate_percentage_change that utilizes the reduce function to iterate over each row in the dataframe, calculate the percentage change for each country, and then compile the results into a new dataframe. The sort_values method should be then used to identify the top five countries with the highest percentage change based on the calculated values.
  

In [13]:
import numpy as np

def calculate_percentage_change(x, y):
    """
    Calculate percentage change from initial value (x) to final value (y).

    Useful in finance, economics, and data analysis to express change as a percentage of the initial value.
    Avoids division by zero by returning NaN for x=0 or any NaN inputs.

    Parameters:
    - x (float or int): Initial value, must not be zero.
    - y (float or int): Final value.

    Returns:
    - float: Percentage change from x to y. Returns NaN for invalid inputs.
    """
    if x == 0 or np.isnan(x) or np.isnan(y):
        return np.nan
    else:
        return ((y - x) / x) * 100
    
    

In [14]:
# Filter for 2007 and 2008
banking_crisis_restricted = banking_crisis_raw[banking_crisis_raw["Year"].isin([2007, 2008])]

# Sort data by country and year
sorted_banking_crisis = banking_crisis_restricted.sort_values(by=["Country Name", "Year"])

# Convert "exch_usd" column to numeric, set errors to NaN
sorted_banking_crisis["exch_usd"] = pd.to_numeric(sorted_banking_crisis["exch_usd"], errors="coerce")

# Calculate percentage change in "exch_usd" for each country
percentage_changes_grouped = sorted_banking_crisis.groupby("Country Name")["exch_usd"].apply(
    lambda x: reduce(calculate_percentage_change, [x.iloc[0], x.iloc[-1]])
).reset_index(name="Percentage Change")

# Identify top 3 countries with the highest percentage change
percentage_changes_grouped = percentage_changes_grouped.sort_values(by="Percentage Change", ascending=False).head(3)

print(percentage_changes_grouped)


      Country Name  Percentage Change
69        Zimbabwe       1.052632e+15
28         Iceland       9.495554e+01
64  United Kingdom       3.742626e+01


## DECORATORS
Decorators in programming, particularly in Python, are used to add functionality to an existing function or method without changing its structure. This is done by wrapping the original function in another function that adds the new functionality, effectively extending or modifying its behavior.

In [15]:
# Let's start with a simple function
def calculate_simple_interest(principal, rate, time):
    """
    Calculate the simple interest for a loan.
    """
    return principal * rate * time

Decorator to Adjust for Inflation

Now, we'll define a decorator that takes the calculated simple interest and adjusts it for inflation. This example assumes a fixed inflation rate for simplicity.

In [16]:
def adjust_for_inflation(func):
    """
    A decorator that adjusts the calculated simple interest by accounting for inflation.
    The inflation rate is assumed to be 2% per annum for this example.
    """
    inflation_rate = 0.02  # 2% inflation rate
    
    def wrapper(principal, rate, time):
        # Calculate the nominal interest using the original function
        nominal_interest = func(principal, rate, time)
        # Adjust the interest for inflation
        real_interest = nominal_interest / ((1 + inflation_rate) ** time)
        return real_interest
    return wrapper

We apply the adjust_for_inflation decorator to the calculate_simple_interest function to automatically adjust the interest calculation for inflation.

In [17]:
@adjust_for_inflation
def calculate_simple_interest(principal, rate, time):
    return principal * rate * time
real_interest = calculate_simple_interest(100, 0.05, 3)
print(f"Real Interest after adjusting for inflation {real_interest:.2f}%")

Real Interest after adjusting for inflation 14.13%


Let's use our decorated function to calculate the real interest, taking into account inflation, and observe how the new decorator modifies the behavior of the original function.

Decorator Function: adjust_for_inflation wraps around calculate_simple_interest to adjust its output. The decorator assumes a constant inflation rate.

Wrapper Function: Inside the decorator, the wrapper function calculates the nominal interest by calling the original calculate_simple_interest function. It then adjusts this nominal interest to account for inflation over the specified time period, using a simple formula to calculate the real interest.

Applying the Decorator: By applying @adjust_for_inflation, every call to calculate_simple_interest now returns the real interest adjusted for inflation, demonstrating how decorators can add significant functionality to existing functions without altering their core logic.

Let me walk you through, step-by-step, a function with decorators that involves calculating the price of a product, adjusting for tax, and then applying a seasonal discount. We'll chain decorators to add these functionalities progressively.

Step 1: Define a function called "base_price" that simply multiplies the quantity of items by the unit price to find the base price. 

In [18]:
def base_price(quantity, unit_price):
    """
    Calculate the base price of a product given the quantity and unit price.
    """
    return quantity * unit_price

Step 2: Add a Decorator to Adjust for Tax
Now, let's define a decorator that takes a tax rate and applies it to the base price calculated by the original function.

In [19]:
def apply_tax(tax_rate):
    """
    Decorator that applies a tax rate to the price of a product.
    """
    def decorator(func):
        def wrapper(quantity, unit_price):
            print("1")
            base = func(quantity, unit_price)
            taxed_price = base * (1 + tax_rate)
            return taxed_price
        return wrapper
    return decorator

@apply_tax(tax_rate=0.07)  # Assume a 7% tax rate
def base_price(quantity, unit_price):
    """
    Calculate the base price of a product given the quantity and unit price.
    """
    return quantity * unit_price

# This decorator adjusts the base price by adding a tax rate to it. Next, we will introduce another layer of modification 
# by applying a seasonal discount through another decorator.

Step 3: Add Another Decorator for Seasonal Discount
Finally, we add a decorator to apply a seasonal discount to the already taxed price. 
This demonstrates how decorators can be chained to sequentially apply multiple modifications or enhancements.

In [20]:
def seasonal_discount(discount_rate):
    """
    Decorator that applies a seasonal discount rate to the product's price.
    """
    def decorator(func):
        def wrapper(quantity, unit_price):
            print("2")
            price_after_tax = func(quantity, unit_price)
            discounted_price = price_after_tax * (1 - discount_rate)
            return discounted_price
        return wrapper
    return decorator

@seasonal_discount(discount_rate=0.1)  # Apply a 10% seasonal discount
@apply_tax(tax_rate=0.07)  # Apply a 7% tax rate
def base_price(quantity, unit_price):
    """
    Calculate the base price of a product given the quantity and unit price.
    """
    return quantity * unit_price

This code calculates the price of a product by first calculating the base price, then adjusting for tax, and finally applying a seasonal discount. Each decorator adds a layer of functionality to the original base_price calculation, demonstrating the power of decorators for composing behaviors in a clean and modular way. This approach allows for flexibility in applying or removing adjustments without altering the core business logic.


To call the base_price function that has been decorated with both the seasonal_discount and apply_tax decorators, you simply need to provide the required arguments: quantity and unit_price. These arguments represent the number of items you're calculating the price for and the cost per item, respectively.

Here's how you can call the base_price function:

In [21]:
# Example of calling the decorated base_price function
quantity = 10  # For example, 10 units of the product
unit_price = 50  # Assume each unit costs $50

final_price = base_price(quantity, unit_price)

print(f"Final price after tax and discount: ${final_price:.2f}")

2
1
Final price after tax and discount: $481.50


This call takes into account the sequential application of the decorators: first, the tax is applied to the base price, and then the discounted rate is applied to the already taxed price, reflecting the final sale price of the product.

the @ statements for decorators matters significantly because decorators are applied from the bottom up. This order determines how the function is wrapped and, consequently, how its behavior is modified or extended.
:

Bottom-Up Application: When you stack decorators, the decorator closest to the function (i.e., the one directly above it) is applied first. After that, the next decorator up is applied, and so on. The last decorator (the one at the top) is applied last.

Effect on Functionality: The order can change the result of the function because each decorator potentially modifies the input, output, or behavior of the function it wraps. The modifications made by one decorator are passed on to the next decorator in the stack.

Q12: 
Rewrite this function and change the order of the decorator. Try to do it without copy-paste so that your fingers help you think thorugh the logic. How does it change the behavior? Please answer using words.

In [22]:
@apply_tax(tax_rate=0.07)
@seasonal_discount(discount_rate=0.1)
def base_price(quantity, unit_price):
    """
    Calculate the base price of a product given the quantity and unit price.
    """
    return quantity * unit_price

final_price = base_price(quantity, unit_price)

print(f"Final price after tax and discount: ${final_price:.2f}")

1
2
Final price after tax and discount: $481.50


<span style="color:blue"> I added the print function to the decorator functions to demonstrate that the decorator closest to the function is applied first. In this case, the result does not change because the expression price * (1 - discount_rate) * (1 + tax_rate) yields the same result as price * (1 + tax_rate) * (1 - discount_rate), due to the commutative property of multiplication.
</span>


## CLASSES IN PYTHON

Python classes are a fundamental part of object-oriented programming (OOP), allowing for the encapsulation of data and functions that operate on that data in a single entity, known as an object. What does that mean? It means you can create code that can put a nice package around all of the functions you're writing so that they are neat, tidy and not strewn all about the place. 

In the context of global affairs, Python classes can be used to model complex entities and relationships found in international relations, economics, environmental studies, and more. By defining classes, you can create more organized, scalable, and reusable code for analyses, simulations, data processing and visualization.

**Basic Concepts of Python Classes**

*Class*: A blueprint for creating objects. A class defines a set of attributes and methods that are common to all objects instantiated from this class.

*Object*: An instance of a class. Each object can have unique values for the attributes defined by its class.
Attribute: A variable that belongs to an object or class. In the context of global affairs, an attribute could represent data like a country's GDP, population, or carbon emissions.

*Method*: A function that belongs to an object or class and can access or modify the object's attributes. For example, a method might update a country's economic data or calculate its growth rate.

Let's look at some examples where this will become easier to understand:


In [23]:
class Country:
    def __init__(self, name, population, gdp):
        self.name = name
        self.population = population
        self.gdp = gdp
    
    def display_info(self):
        print(f"Country: {self.name}\nPopulation: {self.population}\nGDP: ${self.gdp} trillion")

    def gdp_per_capita(self):
        return self.gdp * 1e12 / self.population


In this example, the Country class has three attributes (name, population, gdp) and two methods (display_info and gdp_per_capita). The __init__ method initializes new instances of the class, display_info prints out the country's information, and gdp_per_capita calculates the GDP per capita.

This is how you'd use it:

In [24]:
usa = Country("United States", 331002651, 21.43)
china = Country("China", 1439323776, 14.34)

usa.display_info()
china.display_info()

print(f"GDP per capita (USA): ${usa.gdp_per_capita():.2f}")
print(f"GDP per capita (China): ${china.gdp_per_capita():.2f}")


Country: United States
Population: 331002651
GDP: $21.43 trillion
Country: China
Population: 1439323776
GDP: $14.34 trillion
GDP per capita (USA): $64742.68
GDP per capita (China): $9963.01


**Extending the Model to Include Relationships**
To further apply classes in the context of global affairs, you might define additional classes to represent international organizations, treaties, or global issues, and use methods to model interactions or relationships between countries.

In [25]:
class Treaty:
    def __init__(self, name, member_countries):
        self.name = name
        self.member_countries = member_countries
    
    def add_country(self, country):
        self.member_countries.append(country)
    
    def display_members(self):
        print(f"Treaty: {self.name}\nMembers: {', '.join([country.name for country in self.member_countries])}")


Q13:
Write a class in python that takes country, exchange rate, year, and systemic crisis as parameters and code to calculate the percentage change in exchange rate for two years, and filters for inflation rate 30% or higher.

In [26]:
class CountryExchangeData:
    def __init__(self, excel_path="20160923_global_crisis_data.xlsx"):
        self.data_frame = pd.read_excel(excel_path)
    
    def calculate_exchange_rate_change(self, country, year_start, year_end):
        """
        Calculates the percentage change in exchange rate for a specific country between two given years.

        Parameters:
        - country (str): The name of the country.
        - year_start (int): The starting year.
        - year_end (int): The ending year.

        Returns:
        - float: The percentage change in exchange rate for the specified country from year_start to year_end. 
                 Returns None if the country or either year is not in the dataset.
        """
        # Filter the DataFrame for the specified country
        country_data = self.data_frame[self.data_frame["Country"] == country]
        
        # Check if the country data exists and the specified years are available
        if country_data.empty:
            return None
        elif (year_start not in country_data["Year"].values) or (year_end not in country_data["Year"].values):
            return None
        else:
            # Calculate percentage change in exchange rate
            rate1 = country_data[country_data["Year"] == year_start]["Exchange Rate"].iloc[0]
            rate2 = country_data[country_data["Year"] == year_end]["Exchange Rate"].iloc[0]
            return ((rate2 - rate1) / rate1) * 100
        
    def filter_high_inflation(self, inflation_threshold=30):
        """
        Filters the DataFrame to find rows with high inflation rates.

        Parameters:
        - inflation_threshold (float): The threshold value for defining high inflation rates.
                                       Default is 30.

        Returns:
        - pandas.DataFrame: DataFrame containing rows with inflation rates greater than or equal to the threshold.
        """
        # Rename column and convert "Inflation rate" to numeric
        self.data_frame = self.data_frame.rename(columns={
            "Inflation, Annual percentages of average consumer prices": "Inflation rate"
        })
        self.data_frame["Inflation rate"] = pd.to_numeric(self.data_frame["Inflation rate"], errors="coerce")

        # Filter the DataFrame based on the inflation threshold
        high_inflation_data = self.data_frame[self.data_frame["Inflation rate"] >= inflation_threshold]
        return high_inflation_data


Q14: 
Add an IMF decorator to your class that assumes an IMF intervention that would adjust the inflation rate to a maximum of 18.5%.

In [27]:
def imf_intervention(func):
    def wrapper(*args, **kwargs):
        original_result = func(*args, **kwargs)
        return min(original_result, 18.5) if original_result is not None else None
    return wrapper

class CountryExchangeData:
    def __init__(self, excel_path="20160923_global_crisis_data.xlsx"):
        self.data_frame = pd.read_excel(excel_path)
    
    @imf_intervention
    def calculate_exchange_rate_change(self, country, year_start, year_end):
        """
        Calculates the percentage change in exchange rate for a specific country between two given years.

        Parameters:
        - country (str): The name of the country.
        - year_start (int): The starting year.
        - year_end (int): The ending year.

        Returns:
        - float: The percentage change in exchange rate for the specified country from year_start to year_end. 
                 Returns None if the country or either year is not in the dataset.
        """
        # Filter the DataFrame for the specified country
        country_data = self.data_frame[self.data_frame["Country"] == country]
        
        # Check if the country data exists and the specified years are available
        if country_data.empty:
            return None
        elif (year_start not in country_data["Year"].values) or (year_end not in country_data["Year"].values):
            return None
        else:
            # Calculate percentage change in exchange rate
            rate1 = country_data[country_data["Year"] == year_start]["Exchange Rate"].iloc[0]
            rate2 = country_data[country_data["Year"] == year_end]["Exchange Rate"].iloc[0]
            return ((rate2 - rate1) / rate1) * 100
        
    def filter_high_inflation(self, inflation_threshold=30):
        """
        Filters the DataFrame to find rows with high inflation rates.

        Parameters:
        - inflation_threshold (float): The threshold value for defining high inflation rates.
                                       Default is 30.

        Returns:
        - pandas.DataFrame: DataFrame containing rows with inflation rates greater than or equal to the threshold.
        """
        # Rename column and convert "Inflation rate" to numeric
        self.data_frame = self.data_frame.rename(columns={
            "Inflation, Annual percentages of average consumer prices": "Inflation rate"
        })
        self.data_frame["Inflation rate"] = pd.to_numeric(self.data_frame["Inflation rate"], errors="coerce")

        # Filter the DataFrame based on the inflation threshold
        high_inflation_data = self.data_frame[self.data_frame["Inflation rate"] >= inflation_threshold]
        return high_inflation_data

Q15:
Please confirm that you have access to Anthropic's Claude: https://www.anthropic.com/news/claude-2-1


Your confirmation here: Yes